
# Prueba Transformer

In [1]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.5MB 863kB/s ta 0:00:01


In [20]:
import torch

In [21]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [22]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [35]:
import io
import torch
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

url = 'https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip'
test_filepath, valid_filepath, train_filepath = extract_archive(download_from_url(url))
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer,
                                      iter(io.open(train_filepath,
                                                   encoding="utf8"))))

def data_process(raw_text_iter):
  data = [torch.tensor([vocab[token] for token in tokenizer(item)],
                       dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_data = data_process(iter(io.open(train_filepath, encoding="utf8")))
val_data = data_process(iter(io.open(valid_filepath, encoding="utf8")))
test_data = data_process(iter(io.open(test_filepath, encoding="utf8")))

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data[:5000], batch_size)
val_data = batchify(val_data[:5000], eval_batch_size)
test_data = batchify(test_data[:5000], eval_batch_size)

36718lines [00:00, 36817.75lines/s]


In [36]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [37]:
ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [38]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [39]:
best_val_loss = float("inf")
epochs = 3 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  0.26s | valid loss 12.81 | valid ppl 367431.47
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  0.14s | valid loss  9.78 | valid ppl 17670.10
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  0.11s | valid loss  9.83 | valid ppl 18661.65
-----------------------------------------------------------------------------------------


In [40]:
test_loss = evaluate(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  9.40 | test ppl 12118.95


In [27]:
torch.cuda.is_available()

True

In [22]:
torch.cuda.current_device()

0

In [23]:
torch.cuda.device(0)

In [28]:
torch.cuda.device(3)

In [26]:
torch.cuda.current_stream()

<torch.cuda.Stream device=cuda:0 cuda_stream=0x0>

# ARQUITECTURA TRANSFORMER

In [7]:
d_model = 64 #None#
dff = 128#
nhead = 4#
nlayers = 6#
dropout = 0#

In [8]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class EncoderModel(nn.Module):
    """
    1. With positional encoding and look-ahead masks,
    2. With positional encoding and without look-ahead masks in a bi-directional fashion,
    3. Without positional encoding and without look-ahead masks, which corresponds to modelling
    the points in a stroke as set of points.
    """
    def __init__(self, d_model, nhead, dff, nlayers, dropout = 0.2):
        super(EncoderModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, dff, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.d_model = d_model
        self.init_weights()
        
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        #self.transformer_encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        return output

In [9]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [66]:
EncoderModel(d_model, nhead, dff, nlayers, dropout)

EncoderModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, 

In [1]:
!pip3 install torch==1.7.1
!pip3 install tfrecord

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Created wheel for tfrecord: filename=tfrecord-1.11-py3-none-any.whl size=10785 sha256=a9828ff44e7e6c714df02320fed50f54fba2a2b00e89fba68c48f39d25b7aafc
  Stored in directory: /home/ajimenez/.cache/pip/wheels/17/92/23/6413fae5357b54e6ebcfda2a4a2f6cf086b6e10414581df8d3
Successfully built tfrecord


In [3]:
import json
import torch
from tfrecord.torch.dataset import TFRecordDataset
import tensorflow as tf
path_data = "/data/jcabrera/didi_wo_text/training/"
name_file = "diagrams_wo_text_20200131-00000-of-00010"
tfrecord_path = path_data + name_file
index_path = None

In [4]:
description = {"key": "byte", "label_id":"byte",
               "ink":"float", "stroke_length":"int", "shape":"int", "num_strokes":"int", 
               "rdp_ink":"float", "rdp_stroke_length":"int", 
               "rdp_shape":"int", "rdp_num_strokes":"int",
               }
dataset = TFRecordDataset(tfrecord_path, index_path, description)
loader = torch.utils.data.DataLoader(dataset, batch_size=1)
dataiter = iter(loader)
data = dataiter.next()
print(data)
data["ink"]


{'key': tensor([[ 48,  48,  51,  48,  55,  56,  57,  97,  57,  54,  97,  97, 101,  55,
         101,  97]], dtype=torch.uint8), 'label_id': tensor([[101,  50,  51,  56,  56,  97, 100,  56,  55,  98, 102,  55,  97,  52,
          98,  54,  49,  97,  51,  99,  52,  97,  55,  52, 102,  51,  49,  53,
          48, 101,  51, 100,  98,  97,  57,  56,  49, 100,  49,  97]],
       dtype=torch.uint8), 'ink': tensor([[0.1517, 0.5780, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]), 'stroke_length': tensor([[ 32,  32,  25,  43, 128,  31,  33,  34,  29]], dtype=torch.int32), 'shape': tensor([[  9, 128,   4]], dtype=torch.int32), 'num_strokes': tensor([[9]], dtype=torch.int32), 'rdp_ink': tensor([[187.4000, 751.9100,   0.0000,  ...,   0.0000,   0.0000,   0.0000]]), 'rdp_stroke_length': tensor([[38, 36, 33, 21, 29, 19, 16, 13, 16]], dtype=torch.int32), 'rdp_shape': tensor([[ 9, 38,  4]], dtype=torch.int32), 'rdp_num_strokes': tensor([[9]], dtype=torch.int32)}


/home/ajimenez/.local/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])


tensor([[0.1517, 0.5780, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [13]:
pe = PositionalEncoding(128, 0.1)

In [19]:
pe.forward(data['ink'].reshape(9,4,128))

tensor([[[ 1.6853e-01,  0.0000e+00,  0.0000e+00,  ...,  1.1111e+00,
           0.0000e+00,  2.2222e+00],
         [ 0.0000e+00,  1.1111e+00,  0.0000e+00,  ...,  1.1111e+00,
           0.0000e+00,  1.1111e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  1.1111e+00,
           0.0000e+00,  1.1111e+00],
         [ 0.0000e+00,  1.1111e+00,  0.0000e+00,  ...,  1.1111e+00,
           0.0000e+00,  1.1111e+00]],

        [[ 2.2037e+00,  6.9043e-01,  1.1020e+03,  ...,  1.1111e+00,
           1.2831e-04,  2.2222e+00],
         [ 9.3497e-01,  6.0034e-01,  0.0000e+00,  ...,  1.1111e+00,
           1.2831e-04,  1.1111e+00],
         [ 9.3497e-01,  0.0000e+00,  8.4636e-01,  ...,  1.1111e+00,
           1.2831e-04,  1.1111e+00],
         [ 9.3497e-01,  6.0034e-01,  0.0000e+00,  ...,  0.0000e+00,
           1.2831e-04,  0.0000e+00]],

        [[ 3.2972e+00,  2.4377e-01,  2.1922e+03,  ...,  0.0000e+00,
           2.5662e-04,  0.0000e+00],
         [ 1.0103e+00, -4.6239e-01,  1.0967e+00,  .

In [ ]:
pe.forward(data['ink'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2895, in _run_cell
    return runner(coro)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2987, in run_cell_async
    cell = self.transform_cell(raw_cell)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3104, in transform_cell
    cell = self.input_transformer_manager.transform_cell(raw_cell)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/inputtransformer2.py", line 590, in transform_cell
    lines = self.do_token_transforms(lines)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/inputtransformer2.py", line 575, in do_token_transforms
    changed, lines = self.do_one_toke

--- Logging error ---
Traceback (most recent call last):
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2895, in _run_cell
    return runner(coro)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2987, in run_cell_async
    cell = self.transform_cell(raw_cell)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3104, in transform_cell
    cell = self.input_transformer_manager.transform_cell(raw_cell)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/inputtransformer2.py", line 590, in transform_cell
    lines = self.do_token_transforms(lines)
  File "/home/ajimenez/.local/lib/python3.6/site-packages/IPython/core/inputtransformer2.py", line 575, in do_token_transforms
    changed, li

In [ ]:
data['ink'].shape

In [10]:
data["shape"]

tensor([[  9, 128,   4]], dtype=torch.int32)